## TP2:

Au cours du TP1, nous avons étudié le modèle *Softmax* pour traiter le problème de classification probabiliste. Le but était de présenter deux étapes importantes de l'entraînement : la forward propagation et la mise à jour des paramètres. Le TP2 reprend le modèle Softmax dans un cadre plus général, celui des réseaux de neurones avec couches cachèes.
Dans ce cadre, on peut considérer le modèle Softmax comme un "module" qui prend en entrèe des "features", e.g. les pixels d'une image, et qui donne en sortie une loi de probabilité sur les étiquettes.
Un réseau de neurones est composé de plusieurs modules, transformant simplement les features d'un espace à un autre en fonction des valeurs courantes des paramètres. Ainsi, le but de l'entraînement est d'apprendre les transformations pertinentes, i.e., en modifiant les paramètres, qui permettront de réaliser la tâche associée au module de sortie. En augmentant le nombre de modules (mais aussi de fonctions non-linéaires), on augmente ainsi la complexité du modèle.

Le premier but du TP2 est de programmer les trois étapes essentielles à l'entraînement d'un réseau de neurones : la *forward propagation*, la *backpropagation* et la *mise à jour des paramètres*. Vérifiez que votre modèle fonctionne. Ensuite, vous pourrez comparer les performances de votre réseau de neurones avec celles de votre modèle Softmax de la semaine dernière.

Une fois ces bases réalisées, on va pouvoir ajouter plusieurs fonctions d'activations en plus de la classique *sigmoïde*: les *tanh* et *relu*. Vous pourrez comparer la sigmoïde et la tanh avec la relu, notamment lorsque l'on utilise 2 couches cachées ou plus. Vous pourrez aussi mettre en évidence le phénomène de sur-apprentissage (travaillez avec une petite sous partie des données si nécessaire).
Pour rappel, les fonctions sont:

$$ tanh(x) = \frac{e^{x} - e^{-x}}{e^{x} + e^{-x}}$$

$$ relu(x) = max(0, x) $$

Remarque: La fonction relu est plus instable numériquement que les deux autres. Il est possible qu'il soit nécessaire de réduire le taux d'apprentissage (ou de l'apapter, en le réduisant au fur et à mesure que l'apprentissage progresse) ou de forcer les valeurs de la relu à rester en dessous d'une limite que l'on choisit (*clipping*).

Enfin, on va implémenter la *régularisation*: on ajoutera à la fonction de mise à jour des paramètres les méthodes de régularisation *L1* et *L2*. Il s'agira ensuite de vérifier leur influence sur les courbes d'apprentissage en faisant varier le paramètre $\lambda$.

A faire: 
- Compléter les fonctions:
    - getDimDataset
    - sigmoid
    - forward
    - backward
    - update
    - softmax
    - computeLoss
    - getMiniBatch
- Compléter les fonctions:
    - tanh
    - relu
    - et faire les expériences demandées.
- Compléter les fonctions:
    - updateParams
    - et faire les expériences demandées.
- Envoyer le notebook avec le code complété avant le **13 décembre 2017** à l'adresse **labeau@limsi.fr** accompagné d'un résumé d'un maximum de 6 pages contenant des figures et des analyses rapides des expériences demandées.
- Le résumé doit être succinct et se focaliser uniquement sur les points essentiels reliés à l'entraînement des réseaux de neurones. En plus des résultats, ce document doit décrire les difficultés que vous avez rencontrées et, dans le cas échéant, les solutions utilisées pour les résoudre. Vous pouvez aussi y décrire vos questions ouvertes et proposer une expérience sur MNIST afin d'y répondre.



In [1]:
%matplotlib inline
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import math
import time
from IPython.display import clear_output

if("mnist.pkl.gz" not in os.listdir(".")):
    !wget http://deeplearning.net/data/mnist/mnist.pkl.gz

#####################
# Gestion des données
#####################  
import dataset_loader
train_set, valid_set, test_set = dataset_loader.load_mnist()

def getDimDataset(train_set):
    n_training = len(train_set[0])
    n_feature = len(train_set[0][0])
    n_label = len(set(train_set[1]))
    return n_training, n_feature, n_label

n_training, n_feature, n_label = getDimDataset(train_set)
print(n_training,n_feature,n_label)
########################
# Gestion des paramètres
########################

# Taille de la couche cachée: sous forme de liste, il est possible
# d'utiliser plusieurs couches cachées, avec par exemple [128, 64]
n_hidden =[256]

# Fonction d'activation: à choisir parmi 'sigmoid', 'tanh' et 'relu'
act_func = 'tanh'

# Taille du batch
batch_size = 500

# Taux d'apprentissage:
eta = 0.001

# Nombre d'époques:
n_epoch = 50

50000 784 10


In [2]:
def getMiniBatch(i, batch_size, train_set, one_hot):
    """
    Return a minibatch from the training set and the associated labels
    Inputs: i: the identifier of the minibatch - int
          : batch_size: the number of training examples - int
          : train_set: the training set - ndarray
          : one_hot: the one-hot representation of the labels - ndarray
    Outputs: the minibatch of examples - ndarray
           : the minibatch of labels - ndarray
           : the number of examples in the minibatch - int
    """
    #one_hot_batch = one_hot[:,idx_begin:idx_end]
    idx_begin = i
    idx_end = i+batch_size
    batch = train_set[0][idx_begin:idx_end]
    one_hot=one_hot.transpose()
    one_hot_batch = one_hot[idx_begin:idx_end,:]
    mini_batch_size = batch.shape[0]
    return np.asfortranarray(batch), one_hot_batch, mini_batch_size

In [3]:
def initNetwork(nn_arch, act_func_name):
    """
    Initialize the neural network weights, activation function and return the number of parameters
    Inputs: nn_arch: the number of units per hidden layer -  list of int
          : act_func_name: the activation function name (sigmoid, tanh or relu) - str
    Outputs: W: a list of weights for each hidden layer - list of ndarray
           : B: a list of bias for each hidden layer - list of ndarray
           : act_func: the activation function - function
           : nb_params: the number of parameters  - int
    """

    W,B = [],[]
    sigma = 1.0
    act_func = globals()[act_func_name] # Cast the string to a function
    nb_params = 0

    if act_func_name=='sigmoid':
        sigma = 4.0

    for i in range(np.size(nn_arch)-1):
        w = np.random.normal(loc=0.0, scale=sigma/np.sqrt(nn_arch[i]), size=(nn_arch[i+1],nn_arch[i]))
        W.append(w)
        b = np.zeros((w.shape[0],1))
        if act_func_name=='sigmoid':
            b = np.sum(w,1).reshape(-1,1)/-2.0
        B.append(b)
        nb_params += nn_arch[i+1] * nn_arch[i] + nn_arch[i+1]

    return W,B,act_func,nb_params

In [4]:
########################
# Fonctions d'activation
########################

def sigmoid(z, grad_flag=True):
    """
    Perform the sigmoid transformation to the pre-activation values
    Inputs: z: the pre-activation values - ndarray
          : grad_flag: flag for computing the derivatives w.r.t. z - boolean
    Outputs: y: the activation values - ndarray
           : yp: the derivatives w.r.t. z - ndarray
    """
    y=1.0/(1+np.exp(-1*z))
    yp=y*(1-y)
    return y, yp

# A compléter une fois que la première partie du TP finie:

def tanh(z, grad_flag=True):
    """
    Perform the tanh transformation to the pre-activation values
    Inputs: z: the pre-activation values - ndarray
          : grad_flag: flag for computing the derivatives w.r.t. z - boolean
    Outputs: y: the activation values - ndarray
    """
    y=(np.exp(z)-np.exp(-1*z))/(np.exp(z)+np.exp(-1*z))
    yp=1-np.power(y,2)
    return y, yp


def relu(z, grad_flag=True):
    """
    Perform the relu transformation to the pre-activation values
    Inputs: z: the pre-activation values - ndarray
          : grad_flag: flag for computing the derivatives w.r.t. z - boolean
    Outputs: y: the activation values - ndarray
    """
    y=np.maximum(0,z)
    yp=np.zeros_like(y)
    yp[y>0]=1
    return y, yp

In [5]:
####################
# Création du réseau
####################

### Network Architecture
nn_arch = np.array([n_feature] + n_hidden + [n_label])

### Create the neural network
W, B, act_func, nb_params = initNetwork(nn_arch, act_func)
for w in W:
        print(w.shape)

(256, 784)
(10, 256)


In [6]:
def forward(act_func, W, B, X):
    """
    Perform the forward propagation
    Inputs: act_func: the activation function - function
          : W: the weights - list of ndarray
          : B: the bias - list of ndarray
          : X: the batch - ndarray
    Outputs: Y: a list of activation values - list of ndarray
           : Yp: a list of the derivatives w.r.t. the pre-activation of the activation values - list of ndarray
    """
    a=X
    Yp=[]
    Y=[a]
    for i in range(len(W)-1):
        z=(a.dot(W[i].transpose()))+B[i].transpose()
        y,yp=act_func(z)
        Y.append(y)
        Yp.append(yp)
        a=y
    
    z=(a.dot(W[-1].transpose()))+B[-1].transpose()
    Y.append(z)    
    return Y, Yp
W, B, act_func, nb_params = initNetwork(nn_arch, "sigmoid")
#batch=train_set[0][0:5]
one_hot = np.zeros((n_label,n_training))
one_hot[train_set[1],np.arange(n_training)]=1.
batch,one_hot_batch, mini_batch_size=getMiniBatch(0, 5, train_set, one_hot)
Y, Yp=forward(act_func, W, B, batch)
print([y.shape for y in Y])
print([y.shape for y in Yp])


[(5, 784), (5, 256), (5, 10)]
[(5, 256)]


In [7]:
def softmax(z):
    """
        Perform the softmax transformation to the pre-activation values
        :param z: the pre-activation values
        :type z: ndarray
        :return: the activation values
        :rtype: ndarray
    """
    exps = np.exp(z-np.max(z,axis=1).reshape(-1,1))
    somme_exps = np.sum(exps,axis=1)
    for i in range(somme_exps.shape[0]):
        exps[i]/=somme_exps[i]
    return exps
#out=softmax(z)
#print(out.shape)
#print (out)

In [8]:
def backward(error, W, Yp): 
    """
    Perform the backward propagation
    Inputs: error: the gradient w.r.t. to the last layer - ndarray
          : W: the weights - list of ndarray
          : Yp: the derivatives w.r.t. the pre-activation of the activation functions - list of ndarray
    Outputs: gradb: a list of gradient w.r.t. the pre-activation with this order [gradb_layer1, ..., error] - list of ndarray
    """
    
    gradB = [error]
    for w, yp in zip(reversed(W), reversed(Yp)):
        error = error.dot(w) * yp
        gradB.append(error)
    gradB.reverse()
    return gradB

out = softmax(Y[-1])
### Compute the gradient at the top layer
derror = out - one_hot_batch
gradB=backward(derror, W, Yp)
for b in gradB: print(b.shape)

(5, 256)
(5, 10)


In [9]:
def updateParams(theta, dtheta, eta, regularizer=None, lamda=0.):
    """
    Perform the update of the parameters
    Inputs: theta: the network parameters - ndarray w
          : dtheta: the updates of the parameters - ndarray
          : eta: the step-size of the gradient descent - float
          : regularizer: choice of the regularizer: None, 'L1', or 'L2'
          : lambda: hyperparamater giving the importance of the regularizer - float
    Outputs: the parameters updated - ndarray
    """
    theta=np.copy(theta)
    if regularizer=='L1':
        theta+=lamda*np.abs(theta)
    elif regularizer=='L2':
        theta+=lamda*np.power(theta,2)
    return theta - eta * dtheta

In [10]:
def update(eta, batch_size, W, B, gradB, Y, regularizer, lamda):
    """
    Perform the update of the parameters
    Inputs: eta: the step-size of the gradient descent - float 
          : batch_size: number of examples in the batch (for normalizing) - int
          : W: the weights - list of ndarray
          : B: the bias -  list of ndarray
          : gradB: the gradient of the activations w.r.t. to the loss -  list of ndarray
          : Y: the activation values -  list of ndarray
    Outputs: W: the weights updated -  list of ndarray
           : B: the bias updated -  list of ndarray
    """
    # grad_b should be a vector: object.reshape(-1,1) can be useful
    
    accumulated_gradients_W = [np.zeros_like(w) for w in W]
    for i in range(len(W)):
        for x in range(batch_size):
            accumulated_gradients_W[i] += (gradB[i][x].reshape(-1,1)).transpose().dot(Y[i+1][x])
        W[i]=updateParams(W[i], accumulated_gradients_W[i]/batch_size, eta,regularizer,lamda)
        B[i]=updateParams(B[i], np.sum(gradB[i], axis=0, keepdims=True).transpose()/batch_size, eta,regularizer,lamda)

    return W, B

In [11]:
def computeLoss(act_func, W, B, X, labels):
    """
    Compute the loss value of the current network on the full batch
    Inputs: act_func: the activation function - function
          : W: the weights - list of ndarray
          : B: the bias - list of ndarray
          : X: the batch - ndarray
          : labels: the labels corresponding to the batch
    Outputs: loss: the negative log-likelihood - float
           : accuracy: the ratio of examples that are well-classified - float
    """ 
    ### Forward propagation
    Y, Yp = forward(act_func, W, B, X)
 
    ### Compute the softmax and the prediction
    out = softmax(Y[-1])
    #pred = np.argmax(out,axis=0)
    
    #Loss and Accuracy    
    loss = -np.sum(np.log(out[np.arange(out.shape[0]), labels]))/labels.shape[0]
    accuracy = np.mean((np.argmax(out, axis=1) == labels).astype(np.float, copy=False))
    return loss, accuracy

In [ ]:
import copy
# Data structures for plotting
g_i = []
g_train_loss=[]
g_train_acc=[]
g_valid_loss=[]
g_valid_acc=[]

#############################
### Auxiliary variables
#############################
cumul_time = 0.
n_batch = int(math.ceil(float(n_training)/batch_size))
regularizer = None
lamda = 0.
last_loss=np.inf

# Convert the labels to one-hot vector
one_hot = np.zeros((n_label,n_training))
one_hot[train_set[1],np.arange(n_training)]=1.

print('Architecture = {} | Batch size = {} | Eta = {}'.format(nn_arch, batch_size, eta))

#############################
### Learning process
#############################
for i in range(n_epoch):
    W_copy=copy.deepcopy(W)
    B_copy=copy.deepcopy(B)
    for j in range(n_batch):

        ### Mini-batch creation
        batch, one_hot_batch, mini_batch_size = getMiniBatch(j, batch_size, train_set, one_hot)

        prev_time = time.clock()

        ### Forward propagation
        Y, Yp = forward(act_func, W, B, batch)

        ### Compute the softmax
        out = softmax(Y[-1])
        
        ### Compute the gradient at the top layer
        derror = out - one_hot_batch

        ### Backpropagation
        gradB = backward(derror, W, Yp)

        ### Update the parameters
        W, B = update(eta, batch_size, W, B, gradB, Y, regularizer, lamda)

        exit()
        curr_time = time.clock()
        cumul_time += curr_time - prev_time

    ### Training accuracy
    train_loss, train_accuracy = computeLoss(act_func, W, B, train_set[0], train_set[1]) 

    ### Valid accuracy
    valid_loss, valid_accuracy = computeLoss(act_func, W, B, valid_set[0], valid_set[1])
    
    if (train_loss<last_loss):
        eta*=1.2
    else:
        eta*=0.8
        W=W_copy
        B=B_copy
    last_loss=train_loss

    g_i = np.append(g_i, i)
    g_train_loss = np.append(g_train_loss, train_loss)
    g_train_acc = np.append(g_train_acc, train_accuracy)
    g_valid_loss = np.append(g_valid_loss, valid_loss)
    g_valid_acc = np.append(g_valid_acc, valid_accuracy)
    
    print('{:02d}/{} {:.2f}s Tloss={:.4} Taccu={:.4} Vloss={:.4} Vaccu={:.4} Eta={}'.format(i+1, n_epoch, cumul_time, train_loss, train_accuracy, valid_loss, valid_accuracy, eta))
    sys.stdout.flush() # Force emptying the stdout buffer

Architecture = [784 256  10] | Batch size = 500 | Eta = 0.001
01/50 26.63s Tloss=3.195 Taccu=0.1217 Vloss=3.206 Vaccu=0.1157 Eta=0.0012
02/50 53.48s Tloss=3.058 Taccu=0.1292 Vloss=3.084 Vaccu=0.1254 Eta=0.0014399999999999999
03/50 80.41s Tloss=2.866 Taccu=0.1291 Vloss=2.889 Vaccu=0.1315 Eta=0.0017279999999999997
04/50 107.00s Tloss=2.805 Taccu=0.1264 Vloss=2.824 Vaccu=0.1314 Eta=0.0020735999999999997
05/50 133.71s Tloss=2.776 Taccu=0.1245 Vloss=2.792 Vaccu=0.1316 Eta=0.0024883199999999996
06/50 160.44s Tloss=2.755 Taccu=0.1231 Vloss=2.771 Vaccu=0.1323 Eta=0.0029859839999999993
07/50 187.26s Tloss=2.738 Taccu=0.1223 Vloss=2.752 Vaccu=0.1323 Eta=0.003583180799999999
08/50 214.02s Tloss=2.72 Taccu=0.1221 Vloss=2.734 Vaccu=0.1307 Eta=0.0042998169599999985
09/50 240.84s Tloss=2.701 Taccu=0.1215 Vloss=2.715 Vaccu=0.1312 Eta=0.005159780351999998
10/50 267.49s Tloss=2.682 Taccu=0.1297 Vloss=2.695 Vaccu=0.1401 Eta=0.0061917364223999976
11/50 294.19s Tloss=2.66 Taccu=0.1371 Vloss=2.673 Vaccu=0.1

In [ ]:
plt.plot(g_i,g_train_loss,label='train_loss')
plt.plot(g_i,g_valid_loss,label='valid_loss')
plt.xlabel("epoch")
plt.ylabel("Negative log-likelihood")
plt.legend()

In [ ]:
plt.plot(g_i,1.0-g_train_acc,label='train_acc')
plt.plot(g_i,1.0-g_valid_acc,label='valid_acc')
plt.xlabel("epoch")
plt.ylabel("Classification error")
plt.ylim([0.,1.])
plt.legend()